In [ ]:
import json
def read_and_parse_json(file_path):
    parsed_objects = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
            # Parsing each line as a separate JSON object
              json_object = json.loads(line.strip())
              parsed_objects.append(json_object)
            except:
              print(f"Error serializing record: {line}")
              continue
    return parsed_objects



In [ ]:
import json

# Open and read the JSON file
with open('liver_cancer_medllama_geo_output.json', 'r') as file:
    data = json.load(file)


print(type(data))
print(len(data))


In [ ]:
data = read_and_parse_json("data/ovarian_cancer_medllama_biosample_output.json")

In [ ]:
def printSample(record, term):
  val_rec = record[term]
  l = ""
  for val in val_rec.keys():
    l = l + str(val) + ' : ' + str(val_rec[val]) + '\n'

  return l

In [ ]:
samples = []
cedar_samples = []
llm_samples = []

for d in data:
  samples.append(d['BioSample'])
  llm_samples.append(d['LLM'])
  cedar_samples.append(d['LLM+CEDAR'])

In [ ]:
def search(samples, field_name, field_value):
  return_set = []
  for ind, sample in enumerate(samples):
    if (field_name in sample.keys()):
      if (sample[field_name] == field_value.lower()):
        return_set.append([ind, sample])
  return return_set

In [ ]:
tissue_lung_biosample = search(samples, "tissue", "blood")
tissue_lung_llm = search(llm_samples, "tissue", "blood")
tissue_lung_cedar = search(cedar_samples, "tissue", "blood")

In [ ]:
print(len(tissue_lung_biosample))
print(len(tissue_lung_llm))
print(len(tissue_lung_cedar))

In [ ]:
tv = {}
for t in cedar_samples:
    if 'tissue' in t:
        count = tv.get(t['tissue'],0)
        tv[t['tissue']] = count + 1
print(tv)

In [ ]:
print("Coverage Ratio")
print(f"BioSample: {len(tissue_lung_biosample)/len(samples)}")
print(f"GPT4: {len(tissue_lung_llm)/len(samples)}")
print(f"GPT4+CEDAR: {len(tissue_lung_cedar)/len(samples)}")

In [ ]:
print(len(samples))

In [ ]:
# Open the file and read lines
present_list = []
next_list = []

file_path = 'annotated_1000_samples_llama.txt'  # Replace with your file path
with open(file_path, 'r') as file:
    for line in file:
        # Split the line based on the colon separator
        parts = line.strip().split(':')
        
        if len(parts) == 3:
            index, present, next_val = parts  # Unpack the parts
            
            # Strip any extra spaces around values
            present = present.strip()
            next_val = next_val.strip()

            # Append values to respective lists
            present_list.append(present)
            next_list.append(next_val)

# Output the two lists
present_list, next_list

In [ ]:
ground_truth = []
with open('annotated_800_simple_ovary.txt', 'r') as file:
        for line in file:
            ground_truth.append(line.strip())
            

In [ ]:
print(len([ground_truth[i] for i in range(800) if ground_truth[i] == 'blood']))

In [ ]:
print(ground_truth)

In [ ]:
def evaluate(samples, ground_truth, val):
    rel = ground_truth.count(val)
    print(rel)
    
    # Indices of relevant items in ground_truth
    grd = []
    for indx, i in enumerate(ground_truth):
        if i == val:
            grd.append(indx)
    
    # Calculate precision
    prec = 0
    for [ind, sample] in samples:
        if ind in grd:
            prec += 1
    
    # Precision and recall calculations
    precision = prec / len(samples) if len(samples) > 0 else 0
    recall = prec / rel if rel > 0 else 0
    
    # F1 score calculation
    if precision + recall > 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
    else:
        f1_score = 0

    # Return precision, recall, and F1 score
    return precision, recall, f1_score
    
    

In [ ]:
print(evaluate(tissue_lung_biosample, ground_truth, 'ovary'))

In [ ]:
print(evaluate(tissue_lung_llm, ground_truth, 'ovary'))

In [ ]:
print(evaluate(tissue_lung_cedar, ground_truth, 'ovary'))